### Dataset: MovieLens
### Algorithm: Time-Aware ItemKNN Incremental Ding 2005
### Padding unknown user/item: RecentPop
### Flow: Library Flow

In [1]:
k = 100
from streamsight.datasets.movielens import MovieLens100K
from streamsight.settings import SlidingWindowSetting
dataset = MovieLens100K()
data = dataset.load()

setting_window = SlidingWindowSetting(
    background_t=880606923,
    window_size=60 * 60 * 24 * 30, # day times N
    top_K=k
)

setting_window.split(data)

INFO - streamsight package loaded.
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

DEBUG - MovieLens100K being initialized with 'data' as the base path.
DEBUG - MovieLens100K is initialized.
INFO - MovieLens100K is loading dataset...
DEBUG - Data file is in memory and in dir specified.
DEBUG - MovieLens100K applying filters set.
DEBUG - 	interactions before preprocess: 100000
DEBUG - 	items before preprocess: 1682
DEBUG - 	users before preprocess: 943
DEBUG - 	interactions after preprocess: 100000
DEBUG - 	items after preprocess: 1682
DEBUG - 	users after preprocess: 943
INFO - MovieLens100K dataset loaded - Took 0.0533s
DEBUG - Splitting data...
DEBUG - Performing lt(t, 2147483647)
DEBUG - Performing lt(t, 880606923)
DEBUG - Performing ge(t, 8806

  0%|          | 0/4 [00:00<?, ?it/s]

DEBUG - NPastInteractionTimestampSplitter(t=880606923,t_lower=None,t_upper=2592000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=880606923
DEBUG - Performing lt(t, 883198923)
DEBUG - Performing ge(t, 880606923)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=880606923,t_lower=None,t_upper=2592000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 880606923 resulted in empty unlabelled testing samples.
DEBUG - NPastInteractionTimestampSplitter(t=880606923,t_lower=None,t_upper=2592000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=883198923
DEBUG - Performing lt(t, 885790923)
DEBUG - Performing ge(t, 883198923)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=883198923,t_lower=None,t_upper=2592000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 883198923 resulted in empty 

5it [00:00, 71.13it/s]               

INFO - Finished split with window size 2592000 seconds. Number of splits: 5 in total.
INFO - SlidingWindowSetting data split - Took 0.0954s
DEBUG - Checking split attribute and sizes.
DEBUG - Checking split attributes.
DEBUG - Split attributes are set.
DEBUG - Checking size of split sets.
DEBUG - Size of split sets are checked.
INFO - SlidingWindowSetting data split complete.


In [2]:
from streamsight.evaluators import EvaluatorStreamerBuilder

builder = EvaluatorStreamerBuilder()
builder.add_setting(setting_window)
builder.set_metric_K(k)
builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
builder.add_metric("NDCGK")
evaluator = builder.build()

In [3]:
from streamsight.algorithms import TARSItemKNNDing
external_model = TARSItemKNNDing(K=k)
external_model_id = evaluator.register_algorithm(algorithm_name="MovieLens_TAItemKNN_Inc_Ding")
print(external_model_id)

INFO - Registering algorithm name MovieLens_TAItemKNN_Inc_Ding with ID: bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d registered
bdd640fb-0667-4ad1-9c80-317fa3b1799d


In [4]:
evaluator.start_stream()

DEBUG - Resetting data generators.
DEBUG - Data generators are reset.
DEBUG - Preparing evaluator for streaming
DEBUG - (user x item) shape defined is (413, 1415)
DEBUG - Shape of dataframe stored in matrix was (39235, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (39235, 5) after masking
DEBUG - Final (user x item) shape defined is (413, 1415)
DEBUG - Caching evaluation data for step 1
DEBUG - (user x item) shape defined is (413, 1415)
DEBUG - Shape of dataframe stored in matrix was (7584, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (7584, 5) after masking
DEBUG - Final (user x item) shape defined is (413, 1415)
DEBUG - (user x item) shape defined is (413, 1415)
DEBUG - Shape of dataframe stored in matrix was (7584, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (7500, 5) after masking
DEBUG - Final (user x item) shape defined is (501, 1415)
DEBUG - Data cached for step 1 complete


# Iteration 1

In [5]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
X after decay:  [[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]]
DEBUG - Fitting TARSItemKNNDing complete - Took 0.163s


       interactionid  uid   iid         ts  rating
0                  0    0     0  874724710     4.0
1                  1    0     1  874724727     4.0
2                  2    0     2  874724754     4.0
3                  3    0     3  874724781     4.0
4                  4    0     4  874724843     4.0
...              ...  ...   ...        ...     ...
39230          39230  412   556  880606760     4.0
39231          39231  412  1012  880606818     2.0
39232          39232  412    88  880606874     4.0
39233          39233  412   317  880606874     1.0
39234          39234  412    80  880606874     4.0

[39235 rows x 5 columns]

In [6]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[1.89468502e-30 3.15453537e-14 1.35456346e-14 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.28859055e-11 1.93799489e-11 1.33738069e-11 ... 1.09571945e-25
  0.00000000e+00 1.61525188e-25]
 [4.94588008e-23 6.44094279e-23 5.45484753e-23 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [4.93653787e+00 1.29081984e+01 7.74352733e+00 ... 1.91397932e-01
  0.00000000e+00 3.59193407e-01]
 [2.74402287e+00 7.50403646e+00 4.11437350e+00 ... 3.50129304e-01
  0.00000000e+00 1.88640057e-01]
 [5.06540687e+00 9.90277788e+00 7.16606768e+00 ... 2.61846238e-01
  0.00000000e+00 4.11173481e+00]]
Max user ID:  501
Max item ID:  0
X.shape:  (413, 1415)
Intended shape:  (501, 1415)
Predict frame:         interactionid  uid  iid         ts  rating
39235          39235  412   -1  880606923     3.0
39236          39236  412   -1  880606923    

       interactionid  uid  iid         ts  rating
39235          39235  412   -1  880606923     3.0
39236          39236  412   -1  880606923     2.0
39237          39237  412   -1  880606947     1.0
39238          39238  412   -1  880606993     4.0
39239          39239  412   -1  880606993     5.0
...              ...  ...  ...        ...     ...
50929          50929  500   -1  883195456     5.0
50930          50930  500   -1  883195456     5.0
50931          50931  500   -1  883195479     5.0
50932          50932  500   -1  883195527     5.0
50933          50933  500   -1  883195527     5.0

[7584 rows x 5 columns]

In [7]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 453451 stored elements and shape (501, 1415)>
  Coords	Values
  (0, 94)	8.811747466181525e-15
  (0, 171)	9.384639768844727e-15
  (0, 240)	8.912744043008294e-15
  (0, 270)	8.239874726303845e-15
  (0, 359)	8.933294552289875e-15
  (0, 527)	8.562692143472562e-15
  (0, 579)	9.195623871193698e-15
  (0, 597)	1.0511779209876511e-14
  (0, 628)	8.649625256946032e-15
  (0, 639)	8.453933093081328e-15
  (0, 648)	8.043248725989728e-15
  (0, 725)	8.039965097060887e-15
  (0, 726)	9.090527575399193e-15
  (0, 984)	8.362069156844378e-15
  (0, 1000)	8.057891462898895e-15
  (0, 1061)	8.37496364277176e-15
  (0, 1128)	8.381165023586515e-15
  (0, 1149)	8.362069156844378e-15
  (0, 1172)	9.50334786402973e-15
  (0, 1175)	8.080468955910393e-15
  (0, 1222)	9.00454654696824e-15
  (0, 1249)	1.0049956371326112e-14
  (0, 1255)	8.703517524493689e-15
  (0, 1336)	9.212460007048936e-15
  (0, 1338)	1.1652942590194195e-14
  :	:
  (500, 1390)	0.0
  (500, 1391)	0.0

# Iteration 2

In [8]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (501, 1470)
DEBUG - Shape of dataframe stored in matrix was (11699, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (11699, 5) after masking
DEBUG - Final (user x item) shape defined is (501, 1470)
DEBUG - Caching evaluation data for step 2
DEBUG - (user x item) shape defined is (501, 1470)
DEBUG - Shape of dataframe stored in matrix was (9335, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (9335, 5) after masking
DEBUG - Final (user x item) shape defined is (501, 1470)
DEBUG - (user x item) shape defined is (501, 1470)
DEBUG - Shape of dataframe stored in matrix was (9335, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (9141, 5) after masking
DEBUG - Final (user x item) shape defined is (633, 1470)
DEBUG - Data cached for step 2 complete
X after decay:  [[1. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0

       interactionid  uid  iid         ts  rating
39235          39235  412  262  880606923     3.0
39236          39236  412  407  880606923     2.0
39237          39237  412  848  880606947     1.0
39238          39238  412  337  880606993     4.0
39239          39239  412  583  880606993     5.0
...              ...  ...  ...        ...     ...
50929          50929  500   62  883195456     5.0
50930          50930  500  242  883195456     5.0
50931          50931  500   65  883195479     5.0
50932          50932  500  914  883195527     5.0
50933          50933  500  202  883195527     5.0

[11699 rows x 5 columns]

In [9]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[3.03416788e-10 8.40223180e-09 6.68645486e-09 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.98484002e-28 9.89491029e-01 8.51805430e-01 ... 3.67608072e-01
  3.67608072e-01 4.10230804e-01]
 [8.47125791e-44 6.48721568e-36 5.37281100e-36 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.35914257e-02 2.80549195e-01 1.86872209e-01 ... 3.50856233e-02
  3.50856233e-02 2.48092821e-02]
 [0.00000000e+00 2.80123122e-01 1.10471563e-01 ... 0.00000000e+00
  0.00000000e+00 8.94996586e-02]
 [7.37570018e-01 1.97338797e+00 1.30957944e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
Max user ID:  633
Max item ID:  0
X.shape:  (501, 1470)
Intended shape:  (633, 1470)
Predict frame:         interactionid  uid  iid         ts  rating
50934          50934  501   -1  883215358     3.0
50935          50935  501   -1  883215406    

       interactionid  uid  iid         ts  rating
50934          50934  501   -1  883215358     3.0
50935          50935  501   -1  883215406     4.0
50936          50936  501   -1  883215406     2.0
50937          50937  501   -1  883215525     4.0
50938          50938  501   -1  883215586     5.0
...              ...  ...  ...        ...     ...
64471          64471  490   -1  885784518     4.0
64472          64472  490   -1  885784558     3.0
64473          64473  490   -1  885784558     4.0
64474          64474  490   -1  885784558     4.0
64475          64475  490   -1  885784583     3.0

[9335 rows x 5 columns]

In [10]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 592791 stored elements and shape (633, 1470)>
  Coords	Values
  (0, 94)	9.131287164748539e-28
  (0, 171)	8.77281939559154e-28
  (0, 270)	8.644248984911775e-28
  (0, 359)	8.648220613416716e-28
  (0, 579)	8.786585171025568e-28
  (0, 597)	1.0421577742376045e-27
  (0, 605)	8.8621498545023e-28
  (0, 628)	8.379263780601323e-28
  (0, 725)	8.796394578281803e-28
  (0, 726)	8.564660966722296e-28
  (0, 733)	9.07571449250823e-28
  (0, 787)	8.369758089758425e-28
  (0, 1061)	9.625036136877573e-28
  (0, 1128)	8.796394578281803e-28
  (0, 1172)	9.773771753646447e-28
  (0, 1222)	9.90323132485085e-28
  (0, 1249)	8.595572449916322e-28
  (0, 1265)	8.421906619767879e-28
  (0, 1332)	8.733829087166693e-28
  (0, 1417)	9.2257364656979e-28
  (0, 104)	1.0164546051379178e-27
  (0, 380)	1.1392321517530541e-27
  (0, 390)	1.1715273213135905e-27
  (0, 404)	1.0672504447225812e-27
  (0, 422)	1.1945191118603148e-27
  :	:
  (632, 1445)	0.0
  (632, 1446)	0.0
  (

# Iteration 3

In [11]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (633, 1562)
DEBUG - Shape of dataframe stored in matrix was (13542, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (13542, 5) after masking
DEBUG - Final (user x item) shape defined is (633, 1562)
DEBUG - Caching evaluation data for step 3
DEBUG - (user x item) shape defined is (633, 1562)
DEBUG - Shape of dataframe stored in matrix was (6688, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (6688, 5) after masking
DEBUG - Final (user x item) shape defined is (633, 1562)
DEBUG - (user x item) shape defined is (633, 1562)
DEBUG - Shape of dataframe stored in matrix was (6688, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (6625, 5) after masking
DEBUG - Final (user x item) shape defined is (712, 1562)
DEBUG - Data cached for step 3 complete
X after decay:  [[1. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0

       interactionid  uid   iid         ts  rating
50934          50934  501   119  883215358     3.0
50935          50935  501   301  883215406     4.0
50936          50936  501   303  883215406     2.0
50937          50937  501  1332  883215525     4.0
50938          50938  501   305  883215586     5.0
...              ...  ...   ...        ...     ...
64471          64471  490   299  885784518     4.0
64472          64472  490   898  885784558     3.0
64473          64473  490   349  885784558     4.0
64474          64474  490   249  885784558     4.0
64475          64475  490   917  885784583     3.0

[13542 rows x 5 columns]

In [12]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[2.88493264e-13 9.03986662e-13 1.52094479e-12 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.11912533e-41 3.95707061e-06 5.53399598e-09 ... 9.78655306e-52
  9.78655306e-52 0.00000000e+00]
 [8.42248102e-57 6.55507644e-49 5.43001799e-49 ... 4.01439798e-49
  4.01439790e-49 0.00000000e+00]
 ...
 [0.00000000e+00 1.31072773e+00 2.16936548e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.10203638e+00 5.20686095e+00 5.71758418e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.29398317e+00 7.02385069e+00 4.37592414e+00 ... 0.00000000e+00
  0.00000000e+00 5.30027313e-01]]
Max user ID:  712
Max item ID:  0
X.shape:  (633, 1562)
Intended shape:  (712, 1562)
Predict frame:         interactionid  uid  iid         ts  rating
64476          64476   47   -1  885804123     3.0
64477          64477   47   -1  885805093    

       interactionid  uid  iid         ts  rating
64476          64476   47   -1  885804123     3.0
64477          64477   47   -1  885805093     3.0
64478          64478   47   -1  885805554     5.0
64479          64479  633   -1  885806710     3.0
64480          64480  633   -1  885806710     4.0
...              ...  ...  ...        ...     ...
75252          75252  711   -1  888351713     4.0
75253          75253  711   -1  888351713     3.0
75254          75254  711   -1  888351813     3.0
75301          75301  372   -1  888364663     5.0
75302          75302  372   -1  888364663     5.0

[6688 rows x 5 columns]

In [13]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 632594 stored elements and shape (712, 1562)>
  Coords	Values
  (0, 1003)	1.9859898334481978e-13
  (0, 1337)	2.047539572944139e-13
  (0, 1348)	2.1266182921963593e-13
  (0, 1387)	1.7352737529957743e-13
  (0, 1420)	2.279312392297754e-13
  (0, 1454)	3.0097193784955907e-13
  (0, 1455)	2.1266182921963593e-13
  (0, 1456)	2.639698516773325e-13
  (0, 1457)	3.0490637853643894e-13
  (0, 1469)	3.089992647390814e-13
  (0, 1474)	2.1687336339745192e-13
  (0, 1487)	1.9595862993809584e-13
  (0, 1492)	1.9901664728321376e-13
  (0, 1498)	3.071309359416208e-13
  (0, 1500)	3.0670525183414517e-13
  (0, 94)	7.764983490418059e-41
  (0, 171)	8.467312471144101e-41
  (0, 240)	7.953655124457994e-41
  (0, 262)	7.921758720996719e-41
  (0, 265)	1.0448828233223745e-40
  (0, 597)	9.350278310262983e-41
  (0, 725)	7.757859164124269e-41
  (0, 726)	8.031896431944732e-41
  (0, 733)	7.996694378968603e-41
  (0, 1172)	1.010039430494082e-40
  :	:
  (711, 1537)	0.0
 

# Iteration 4

In [14]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (712, 1597)
DEBUG - Shape of dataframe stored in matrix was (10827, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (10827, 5) after masking
DEBUG - Final (user x item) shape defined is (712, 1597)
DEBUG - Caching evaluation data for step 4
DEBUG - (user x item) shape defined is (712, 1597)
DEBUG - Shape of dataframe stored in matrix was (5950, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (5950, 5) after masking
DEBUG - Final (user x item) shape defined is (712, 1597)
DEBUG - (user x item) shape defined is (712, 1597)
DEBUG - Shape of dataframe stored in matrix was (5950, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (5910, 5) after masking
DEBUG - Final (user x item) shape defined is (784, 1597)
DEBUG - Data cached for step 4 complete
X after decay:  [[1. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0

       interactionid  uid   iid         ts  rating
64476          64476   47   507  885804123     3.0
64477          64477   47   296  885805093     3.0
64478          64478   47  1084  885805554     5.0
64479          64479  633   995  885806710     3.0
64480          64480  633     1  885806710     4.0
...              ...  ...   ...        ...     ...
75298          75298  698   624  888360532     2.0
75299          75299  698   782  888360532     3.0
75300          75300  698   539  888360532     4.0
75301          75301  372  1500  888364663     5.0
75302          75302  372  1513  888364663     5.0

[10827 rows x 5 columns]

In [15]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[5.72739211e-26 1.02947551e-25 1.37663269e-25 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.42621339e-54 1.09408454e-09 4.60220360e-10 ... 7.82683679e-65
  0.00000000e+00 0.00000000e+00]
 [8.73761339e-70 7.15280213e-62 5.97391019e-62 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.92668551e-01 4.37187793e+00 2.35217029e+00 ... 7.04400724e-02
  1.80926258e-01 1.80926258e-01]
 [1.74674342e+00 9.67405886e+00 5.40835190e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.99257934e-01 2.89000769e+00 1.50634654e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
Max user ID:  784
Max item ID:  0
X.shape:  (712, 1597)
Intended shape:  (784, 1597)
Predict frame:         interactionid  uid  iid         ts  rating
75303          75303   48   -1  888396828     4.0
75304          75304   48   -1  888396828    

       interactionid  uid  iid         ts  rating
75303          75303   48   -1  888396828     4.0
75304          75304   48   -1  888396828     4.0
75305          75305   48   -1  888396828     2.0
75306          75306   48   -1  888396966     5.0
75307          75307   48   -1  888396967     3.0
...              ...  ...  ...        ...     ...
83432          83432  679   -1  890939697     2.0
83433          83433  679   -1  890939697     2.0
83434          83434  679   -1  890939753     2.0
83435          83435  679   -1  890939815     3.0
83436          83436  433   -1  890956632     3.0

[5950 rows x 5 columns]

In [16]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 687209 stored elements and shape (784, 1597)>
  Coords	Values
  (0, 1003)	2.38288126118186e-26
  (0, 1337)	2.3855741981765097e-26
  (0, 1454)	3.5072852512700497e-26
  (0, 1455)	2.4710463134101032e-26
  (0, 1456)	3.0212649617365526e-26
  (0, 1469)	3.425391812948924e-26
  (0, 1474)	2.4791614298802607e-26
  (0, 1498)	3.216146137099905e-26
  (0, 171)	9.548863849192379e-54
  (0, 240)	8.759034630739923e-54
  (0, 597)	1.0084129822139403e-53
  (0, 733)	8.935290415160178e-54
  (0, 1172)	1.157205569511793e-53
  (0, 1346)	3.14420385882311e-26
  (0, 1348)	2.5956167176538605e-26
  (0, 1457)	3.310887282288686e-26
  (0, 1467)	4.0329023784099325e-26
  (0, 1500)	3.7858028104829926e-26
  (0, 104)	1.0898259238458327e-53
  (0, 258)	1.126779617299385e-53
  (0, 337)	1.049965203514606e-53
  (0, 422)	1.275456389573426e-53
  (0, 428)	1.376069001292994e-53
  (0, 454)	1.0297557486797142e-53
  (0, 498)	1.1211887568956468e-53
  :	:
  (783, 1572)	0.0
  (

# Iteration 5

In [17]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (784, 1626)
DEBUG - Shape of dataframe stored in matrix was (8134, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (8134, 5) after masking
DEBUG - Final (user x item) shape defined is (784, 1626)
DEBUG - Caching evaluation data for step 5
DEBUG - (user x item) shape defined is (784, 1626)
DEBUG - Shape of dataframe stored in matrix was (11204, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (11204, 5) after masking
DEBUG - Final (user x item) shape defined is (784, 1626)
DEBUG - (user x item) shape defined is (784, 1626)
DEBUG - Shape of dataframe stored in matrix was (11204, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (11090, 5) after masking
DEBUG - Final (user x item) shape defined is (943, 1626)
DEBUG - Data cached for step 5 complete
X after decay:  [[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [0. 0.

       interactionid  uid   iid         ts  rating
75303          75303   48   995  888396828     4.0
75304          75304   48  1572  888396828     4.0
75305          75305   48  1464  888396828     2.0
75306          75306   48  1592  888396966     5.0
75307          75307   48  1585  888396967     3.0
...              ...  ...   ...        ...     ...
83432          83432  679   319  890939697     2.0
83433          83433  679   995  890939697     2.0
83434          83434  679   850  890939753     2.0
83435          83435  679  1373  890939815     3.0
83436          83436  433  1346  890956632     3.0

[8134 rows x 5 columns]

In [18]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[1.03909121e-38 2.76936019e-12 3.31072997e-12 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00661936e-01 3.97891290e-01 2.37677897e-01 ... 6.90215250e-78
  0.00000000e+00 7.08608097e-78]
 [1.26017143e-81 6.70450625e-75 5.58991279e-75 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.85070285e-02 1.30246228e-01 8.42618121e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.20314308e-01 1.84450933e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.16822969e+00 5.39684520e+00 4.10529020e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
Max user ID:  943
Max item ID:  0
X.shape:  (784, 1626)
Intended shape:  (943, 1626)
Predict frame:         interactionid  uid  iid         ts  rating
83437          83437    7   -1  890979062     4.0
83438          83438    7   -1  890979199    

       interactionid  uid  iid         ts  rating
83437          83437    7   -1  890979062     4.0
83438          83438    7   -1  890979199     4.0
83439          83439    7   -1  890979470     3.0
83440          83440    7   -1  890979634     2.0
83441          83441    7   -1  890979947     4.0
...              ...  ...  ...        ...     ...
99995          99995  942   -1  893286638     4.0
99996          99996  942   -1  893286638     3.0
99997          99997  942   -1  893286638     3.0
99998          99998  942   -1  893286638     4.0
99999          99999  942   -1  893286638     4.0

[11204 rows x 5 columns]

In [19]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 888025 stored elements and shape (943, 1626)>
  Coords	Values
  (0, 1589)	2.4997069594064653e-39
  (0, 730)	5.478295405355446e-13
  (0, 1003)	5.012127080362496e-13
  (0, 1008)	5.096241874534071e-13
  (0, 1128)	4.063293501299074e-13
  (0, 1149)	5.380604269137459e-13
  (0, 1265)	5.588962323288997e-13
  (0, 1267)	4.1343213129298457e-13
  (0, 1336)	4.941459113394686e-13
  (0, 1342)	5.135901942520731e-13
  (0, 1347)	4.2154593691666887e-13
  (0, 1364)	4.140696523058935e-13
  (0, 1365)	4.459211640217314e-13
  (0, 1420)	6.797100207268157e-13
  (0, 1455)	4.663190366585798e-13
  (0, 1456)	6.940910864422678e-13
  (0, 1464)	7.183879884077027e-13
  (0, 1465)	4.4663156938533906e-13
  (0, 1466)	4.151300116664212e-13
  (0, 1471)	5.096241874534074e-13
  (0, 1474)	5.451480288862716e-13
  (0, 1476)	5.810609745704914e-13
  (0, 1478)	7.016977912507494e-13
  (0, 1480)	4.910861549149492e-13
  (0, 1484)	4.847674136581044e-13
  :	:
  (942, 1601)	0.0

# evaluate metrics

In [20]:
evaluator.get_all_algorithm_status()

{'MovieLens_TAItemKNN_Inc_Ding_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.COMPLETED: 'COMPLETED'>}

In [21]:
evaluator.metric_results("macro")

macro_score  \
Algorithm                                          Metric                        
MovieLens_TAItemKNN_Inc_Ding_bdd640fb-0667-4ad1... NDCGK_100          0.272461   
                                                   PrecisionK_100     0.143011   
                                                   RecallK_100        0.331695   

                                                                   num_window  
Algorithm                                          Metric                      
MovieLens_TAItemKNN_Inc_Ding_bdd640fb-0667-4ad1... NDCGK_100                5  
                                                   PrecisionK_100           5  
                                                   RecallK_100              5

In [22]:
evaluator.metric_results("micro")


micro_score  \
Algorithm                                          Metric                       
MovieLens_TAItemKNN_Inc_Ding_bdd640fb-0667-4ad1... NDCGK_100         0.275517   
                                                   PrecisionK_100    0.144721   
                                                   RecallK_100       0.333178   

                                                                   num_user  
Algorithm                                          Metric                    
MovieLens_TAItemKNN_Inc_Ding_bdd640fb-0667-4ad1... NDCGK_100            968  
                                                   PrecisionK_100       968  
                                                   RecallK_100          968

In [23]:
evaluator.metric_results("user")

user_id  \
Algorithm                                          Timestamp   Metric                   
MovieLens_TAItemKNN_Inc_Ding_bdd640fb-0667-4ad1... t=880606923 PrecisionK_100       0   
                                                               PrecisionK_100       1   
                                                               PrecisionK_100       7   
                                                               PrecisionK_100       8   
                                                               PrecisionK_100       9   
...                                                                               ...   
                                                   t=890974923 NDCGK_100          468   
                                                               NDCGK_100          470   
                                                               NDCGK_100          909   
                                                               NDCGK_100          490   
                                                               NDCGK_100          498   

                                                                                  score  
Algorithm                                          Timestamp   Metric                    
MovieLens_TAItemKNN_Inc_Ding_bdd640fb-0667-4ad1... t=880606923 PrecisionK_100       0.0  
                                                               PrecisionK_100      0.05  
                                                               PrecisionK_100      0.03  
                                                               PrecisionK_100       0.4  
                                                               PrecisionK_100      0.02  
...                                                                                 ...  
                                                   t=890974923 NDCGK_100       0.643896  
                                                               NDCGK_100       0.157898  
                                                               NDCGK_100       0.250485  
                                                               NDCGK_100            0.0  
                                                               NDCGK_100            0.0  

[2904 rows x 2 columns]

In [24]:
evaluator.metric_results("window")

window_score  \
Algorithm                                          Timestamp   Metric                         
MovieLens_TAItemKNN_Inc_Ding_bdd640fb-0667-4ad1... t=880606923 PrecisionK_100      0.161963   
                                                               RecallK_100         0.347257   
                                                               NDCGK_100           0.284505   
                                                   t=883198923 PrecisionK_100      0.149513   
                                                               RecallK_100         0.309421   
                                                               NDCGK_100           0.268389   
                                                   t=885790923 PrecisionK_100      0.130414   
                                                               RecallK_100         0.323465   
                                                               NDCGK_100           0.254537   
                                                   t=888382923 PrecisionK_100      0.106552   
                                                               RecallK_100         0.302711   
                                                               NDCGK_100           0.230606   
                                                   t=890974923 PrecisionK_100      0.166610   
                                                               RecallK_100         0.375624   
                                                               NDCGK_100           0.324270   

                                                                               num_user  
Algorithm                                          Timestamp   Metric                    
MovieLens_TAItemKNN_Inc_Ding_bdd640fb-0667-4ad1... t=880606923 PrecisionK_100       163  
                                                               RecallK_100          163  
                                                               NDCGK_100            163  
                                                   t=883198923 PrecisionK_100       226  
                                                               RecallK_100          226  
                                                               NDCGK_100            226  
                                                   t=885790923 PrecisionK_100       169  
                                                               RecallK_100          169  
                                                               NDCGK_100            169  
                                                   t=888382923 PrecisionK_100       174  
                                                               RecallK_100          174  
                                                               NDCGK_100            174  
                                                   t=890974923 PrecisionK_100       236  
                                                               RecallK_100          236  
                                                               NDCGK_100            236